# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "4")
         .config("spark.executor.memory", "5g")
         .config("spark.executor.cores", "6")
         .config("spark.driver.memory", "5g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "10g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())


sc = spark.sparkContext
sc.uiWebUrl

21/10/05 20:08:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://69abb479a5e1:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [5]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}


seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.filtrar_sentencias(preprocess=solo_portanto)

estructurales = SentenciasEstructurales(seleccion)

AnalysisException: Path does not exist: file:/home/jovyan/Work/ej/paquetes/nlppen/dataset

# Formar dataset de sentencias estructurales

In [6]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
    'plazosDefinidos' : ArrayType(TimestampType())
}
estructurales.plazosDefinidos(columnas, True)

#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos()

# Cargar procesamiento de sentencias estructurales.

In [7]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
s = estructurales.seleccion.sdf
s.limit(10).toPandas()

# Zona de trabajo (ignorar)

In [9]:
s.filter(s.se_ordena_Ent_Pub.isNotNull()).limit(10).toPandas()
#print(solo_portanto(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'txt']))

In [14]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
df2 = (s
 .where(s.se_ordena_ORG.isNotNull())
 .select(explode('se_ordena_LOC').alias('se_ordena_LOC'), 'anno')
 .groupby('se_ordena_LOC')
 .count()
 .sort(F.desc('count'))
).limit(60).toPandas()
df2

,se_ordena_LOC,count
0,Ministerio de Educación Pública,60
1,Hospital Dr. Rafael Ángel Calderón Guardia,18
2,Alajuela,13
3,Hospital San Rafael de,13
4,Sala,11
5,Ministerio de Trabajo y Seguridad,10
6,Directora del Área de Carrera Docente,9
7,Directora,7
8,Hospital Dr. Rafael Calderón Guardia,7
9,Hospital México,6


In [11]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
df2 = (s
 .where(s.se_ordena_ORG.isNotNull())
 .select(explode('se_ordena_Ent_Pub').alias('se_ordena_Ent_Pub'), 'anno')
 .groupby('se_ordena_Ent_Pub')
 .count()
 .sort(F.desc('count'))
).toPandas()
df2

,se_ordena_Ent_Pub,count
0,Ministerio de Educacion Publica,75
1,Caja Costarricense de Seguro Social,70
2,Direccion General Del Servicio Civil,16
3,Instituto de Desarrollo Agrario,5
4,Universidad Nacional,5
5,Instituto Mixto de Ayuda Social,4
6,Tribunal de Carrera Docente,3
7,Ministerio de Seguridad Publica,3
8,Instituto Costarricense de Electricidad,3
9,Patronato Nacional de La Infancia,3
